In [1]:
import glob
import os
import subprocess
import numpy as np
import pandas as pd
import math

from tqdm import tqdm
from pathlib import Path

import re

# What's happening here?

In this notebook we will read in all the csv files and compile it to useful information. 
We will end up with comprehensive CSV files containing all usefull information. 

Add to this list:
- Responses
- RT
- Testmoment
- Ppt number
- Total time taken


Make list
Check for all instances with that name
if exists: wCam
- get that
else:
- get other



# Get Data

In [2]:
# Declare all directories:

data_dir = Path('Z:/shares/ghepmk_data/2020_Kappen_PMS')

# Create all relevant directories
date_dir = "06102021/"
data_all_dir = os.path.join(data_dir, date_dir)
data_A_dir = os.path.join(data_dir, Path('10001'))
data_B_dir = os.path.join(data_dir, Path('10002'))

# TESTMOMENT A

In [3]:
os.chdir(data_A_dir)
ACsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
ACsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(ACsv):
    ACsvID.append(ACsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in ACsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
ACsvID = set(ACsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [24]:
exclude = ['203', # Double moment A
           '262', # Double moment A
           '285', # Double moment A (both wCAM)
           '334', # Double moment A
           '312', # Double moment A
           '350', # Gives some weird stuff later on
           '361', # Double moment A, but second time moment A also did moment B (2 days later).
           '531', # Double moment A
           '661', # Double moment A, but second time moment A also did moment B.
           '667', # Double moment A, but second time moment A also did moment B. Only 4 days between moment A and B, figure out # i might have been wrong based on video. Almost month later
           
           '105', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '106', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '107', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '111', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '118', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '119', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '123', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '124', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '125', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '126', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '127', # No demographics present. First known participant number is 129. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '999', # No demographics present. Last known participant number is 677. So probably a testing round
           '998', # No demographics present. Last known participant number is 677. So probably a testing round
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           
           '373', # Has no proper experiment order - check in R file
           
           '66_', # Weird participant number, let's see if we can figure out what the correct one would've been here -> change name of csv/webm
           '6_1', # Very hacky, because if the particpiant ends in '6' and date starts with '1' this will also get triggered. Figure out how this ends up in the csv file and try to solve
           '&à&', # Weird stuff, shouldn't be here
           '11_', # Probably the same as the others, check it out
          ]

In [25]:
index = 0
for pptnum in tqdm(ACsvID): # Loop through all file ID's
    print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(ACsv[indices[0]])
            date = re.search(pptnum + '_(.*).csv', ACsv[indices[0]]).group(1)
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(ACsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(ACsv[indices[0]])
                date = re.search(pptnum + '_(.*)_wCAM.csv', ACsv[indices[0]]).group(1)
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe
            # Get deduced indices for startviewing and endviewing
            indicesSTARTView= [x - 2 for x in indices2]
            indicesSTOPView= [x - 1 for x in indices2]
            
            # Make start and endbeep indices
            beepindex = [i for i, s in enumerate(data['trial_type']) if "audio-keyboard-response" in s]
            startBeep = beepindex[0]
            endBeep = beepindex[1]
            
            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"
            
            if statementAB.bool() == True: # If order is A-B, and this is moment A, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment A, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")
            
            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
#                                        [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, [demographics['Isala'][demoindex].values[0]] * stimNum, # With isala
                                       [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, 
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['A'] * stimNum, [date] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], 
                                       data['time_elapsed'][indicesSTARTView], data['time_elapsed'][indicesSTOPView], data['time_elapsed'][indices2], [data['time_elapsed'][startBeep]] * stimNum, [data['time_elapsed'][endBeep]] * stimNum, )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
#                                      'Exclusie', 'TrueFollicular', 'TrueLuteal', 'Isala', # With Isala
                                     'Exclusie', 'TrueFollicular', 'TrueLuteal', 
                                     'Order', 'Moment', 'Date', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt',
                                     'startView', 'stopView', 'stopResponse', 'startBeep', 'endBeep'])
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
#             break
# dataset
datasetA = dataset

  0%|▏                                                                                 | 1/393 [00:00<01:04,  6.05it/s]

Processing: 543
Processing: 339


  1%|▋                                                                                 | 3/393 [00:00<00:55,  7.04it/s]

Processing: 419
Processing: 454


  1%|█                                                                                 | 5/393 [00:00<00:53,  7.29it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 389/393 [00:58<00:00,  5.00it/s]

Processing: 205
Processing: 998
Processing: 674


100%|███████████████████████████████████████████████████████████████████████████████▊| 392/393 [00:58<00:00,  6.13it/s]

Processing: 294
Processing: 644


100%|████████████████████████████████████████████████████████████████████████████████| 393/393 [00:58<00:00,  6.70it/s]


In [22]:
[demographics['Age'][demoindex].values[0]]
[demographics['FirstMenstrual'][demoindex].values[0]]
[demographics['MenstrualDuration'][demoindex].values[0]]
[demographics['Exclusie'][demoindex].values[0]]
[demographics['TrueFollicular'][demoindex].values[0]]
[demographics['TrueLuteal'][demoindex].values[0]]
[demographics['Isala'][demoindex].values[0]]

KeyError: 'Isala'

# TESTMOMENT B

In [27]:
os.chdir(data_B_dir)
BCsv = glob.glob("csv/" + "*ParticipantNo*" + "*.csv", recursive = True) # get all csv's in directory
BCsvID = []
CSVy_n = []
substring = "wCAM" # This is only present in some files but is prefered

for index, item in enumerate(BCsv):
    BCsvID.append(BCsv[index].split("data_{ParticipantNo}_")[1][0:3])
    if substring in BCsv[index]:
        CSVy_n.append(True)
    else:
        CSVy_n.append(False)
        
BCsvID = set(BCsvID)

# demographics = pd.read_csv("Z:/shares/ghepmk_data/2020_Kappen_PMS/24082021/cleanData.csv", encoding = "ISO-8859-1")
demographics = pd.read_csv(data_all_dir + "cleanData.csv", encoding = "ISO-8859-1")

In [28]:
exclude = ['100', # No demographics present. First known participant number is 129.
           '106', # No demographics present. First known participant number is 129. 
           '119', # No demographics present. First known participant number is 129. 
           '125', # No demographics present. First known participant number is 129.
           '126', # No demographics present. First known participant number is 129.
           '127', # No demographics present. First known participant number is 129.
           '778', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           '789', # No demographics present. Last known participant number is 677. So probably wrong participant number. Figure out and then -> change name of csv/webm
           
           
           '350', # Double moment B, but also 1 moment A. Two moment B's 5 days apart. Weird. Figure out
           '361', # Messed it up and made 4_.ipynb crash
           '141', # Double moment B - both wCAM
           '405', # Double moment B
           
           '407', # Double demographics entry (seems like they first filled out with wrong mentrual duration) - figure this out
           '488', # Is 4888, so not yet able to include. Retrace where the typo comes from -> check via date probably
           '459', # Same as above; is 4590
           
           '590', # Kicked out for now, because '4590' is also present (which is wrong). When 4590 is resolved, 590 can come back
           '667', # Kicked out because also '6677' present. So someone did something wrong here. Probably need to kik out 6677 (rename?) -> figure out
           
           
           '14_', # Weird stuff
           '_28', # Weird stuff
           '66_', # Weird stuff
           
           '987', # Trial run probably
           '995', # No demographics, Trial run probably
           '996', # Trial run probably
           '997', # Trial run probably
           '999', # Trial run probably
          ]

In [35]:
index = 0
for pptnum in tqdm(BCsvID): # Loop through all file ID's
    print("Processing: " + pptnum)
    if pptnum not in exclude: # Some participants messed up, so skip those. See earlier cell for descriptions of each
        index += 1
        # Find right file to load and read_csv in
        indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" not in s] # Check if there is a file WITHOUT wCAM because that is preferred (saved later)
        if len(indices) == 1: # if one, that is perfect, read it
            data = pd.read_csv(BCsv[indices[0]])
            date = re.search(pptnum + '_(.*).csv', BCsv[indices[0]]).group(1)
        if len(indices) > 1: # if more than one, something is going wrong, figure out
            print("Too many .csv files for ppt " + pptnum + ".")
            break
        elif len(indices) == 0: # if none, probably only saved before video, so check wCAM
            indices = [i for i, s in enumerate(BCsv) if pptnum in s and "wCAM" in s] # Find instances where files exist WITH wCAM
            if len(indices) == 1: # if one, that is perfect, read it
                data = pd.read_csv(BCsv[indices[0]])
                date = re.search(pptnum + '_(.*)_wCAM.csv', BCsv[indices[0]]).group(1)
            if len(indices) > 1: # if more than one, something is going wrong, figure out
                print("Too many wCAM.csv files for ppt " + pptnum + ".")
                break
            elif len(indices) == 0: # If none, something weird is happening, as it should have appeared here then
                print("No wCAM.csv files for ppt " + pptnum + ".")
                break

        #  Now check whether they completed the experiment
        if not data['stimulus'].str.contains('beep.wav').any(): # if beep.wav is present (final thing in the experiment) they finished
            print("ppt " + pptnum + " did not finish the experiment.")
            break

        # Get data in Demographics file
    #     demoindex = [i for i, s in enumerate(demographics['participantNo'] == int(pptnum)) if s][0]
        demoindex = np.where(demographics['participantNo'] == int(pptnum))[0]
        if len(demoindex) > 1: # Too many entries for this ppt
            print("Too many demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 0:
            print("No demographics entries for ppt " + pptnum + ".")
            break
        elif len(demoindex) == 1:
            # Create indices for the trial selections
            indices = [i for i, s in enumerate(data['trial_type']) if "image-keyboard-response" in s]
            indices2 = [x+1 for x in indices]
            stimNum = len(indices) # to add constant values to dataframe
            # Get deduced indices for startviewing and endviewing
            indicesSTARTView= [x - 2 for x in indices2]
            indicesSTOPView= [x - 1 for x in indices2]
            
            # Make start and endbeep indices
            beepindex = [i for i, s in enumerate(data['trial_type']) if "audio-keyboard-response" in s]
            startBeep = beepindex[0]
            endBeep = beepindex[1]

            # Get correct PSS and BSRI values
            statementAB = demographics['Order'][demoindex].apply(str) == "A-B"
            statementBA = demographics['Order'][demoindex].apply(str) == "B-A"

            if statementAB.bool() == True: # If order is A-B, and this is moment B, then A is luteal
                PSS = demographics['luteaalPSS'][demoindex].values[0]
                BSRI = demographics['luteaalBSRI'][demoindex].values[0]
            elif statementBA.bool() == True: # If order is B-A, and this is moment B, then A is folliculair
                PSS = demographics['folliculairPSS'][demoindex].values[0]
                BSRI = demographics['folliculairBSRI'][demoindex].values[0]
            else:
                print("ppt " + pptnum + " does not have a valid experiment order.")

            df = pd.DataFrame(list(zip([pptnum] * stimNum, [demographics['Age'][demoindex].values[0]] * stimNum, [demographics['FirstMenstrual'][demoindex].values[0]] * stimNum, [demographics['MenstrualDuration'][demoindex].values[0]] * stimNum,
#                                        [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, [demographics['Isala'][demoindex].values[0]] * stimNum,# Isala
                                       [demographics['Exclusie'][demoindex].values[0]] * stimNum, [demographics['TrueFollicular'][demoindex].values[0]] * stimNum, [demographics['TrueLuteal'][demoindex].values[0]] * stimNum, 
                                       [demographics['Order'][demoindex].values[0]] * stimNum, ['B'] * stimNum, [date] * stimNum, [demographics['allSymptoms'][demoindex].values[0]] * stimNum, [demographics['allDisturbance'][demoindex].values[0]] * stimNum, [demographics['allRRS'][demoindex].values[0]] * stimNum,
                                       [demographics['DASS.Total'][demoindex].values[0]] * stimNum, [demographics['DASS.Stress'][demoindex].values[0]] * stimNum, [demographics['DASS.Anxiety'][demoindex].values[0]] * stimNum, [demographics['DASS.Depresh'][demoindex].values[0]] * stimNum,
                                       [demographics['PMSScore'][demoindex].values[0]] * stimNum, [demographics['Contraception'][demoindex].values[0]] * stimNum, [PSS] * stimNum, [BSRI] * stimNum,
                                       data['stimulusShort'][indices], data['valence'][indices2], data['arousal'][indices2], data['rt'][indices2], 
                                       data['time_elapsed'][indicesSTARTView], data['time_elapsed'][indicesSTOPView], data['time_elapsed'][indices2], [data['time_elapsed'][startBeep]] * stimNum, [data['time_elapsed'][endBeep]] * stimNum )),
                           columns =['ID', 'Age', 'FirstMenstrual', 'MenstrualDuration', 
#                                      'Exclusie', 'TrueFollicular', 'TrueLuteal', 'Isala', # Isala
                                     'Exclusie', 'TrueFollicular', 'TrueLuteal', 
                                     'Order', 'Moment', 'Date', 'allSymptoms', 'allDistrubance', 'allRRS', 
                                     'DASS_Total', 'DASS_Stress', 'DASS_Anxiety', 'DASS_Depression', 
                                     'PMSScore', 'Contraception', 'PSS', 'BSRI',
                                     'Stimulus', 'Valence', 'Arousal', 'rt',
                                     'startView', 'stopView', 'stopResponse', 'startBeep', 'endBeep'])
            
        if index == 1:
            dataset = df
        else:
            dataset = pd.concat([dataset, df], ignore_index=True)
    #         break
# dataset
datasetB = dataset

  0%|▏                                                                                 | 1/381 [00:00<01:04,  5.87it/s]

Processing: 543
Processing: 339


  1%|▋                                                                                 | 3/381 [00:00<00:58,  6.41it/s]

Processing: 419
Processing: 454


  1%|▊                                                                                 | 4/381 [00:00<00:57,  6.57it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▎| 378/381 [01:03<00:00,  6.60it/s]

Processing: 674
Processing: 294


100%|███████████████████████████████████████████████████████████████████████████████▊| 380/381 [01:03<00:00,  6.62it/s]

Processing: 273
Processing: 644


100%|████████████████████████████████████████████████████████████████████████████████| 381/381 [01:03<00:00,  5.97it/s]


In [36]:
dataset = pd.concat([datasetA, datasetB], ignore_index=True)
dataset

,ID,Age,FirstMenstrual,MenstrualDuration,Exclusie,TrueFollicular,TrueLuteal,Order,Moment,Date,...,BSRI,Stimulus,Valence,Arousal,rt,startView,stopView,stopResponse,startBeep,endBeep
0,543,39,14,28,NaN,13-5-2021,27-5-2021,A-B,A,13-5-2021,...,110.0,7480.0,100.0,71.0,7842.955001,88253,94256,102102,82917,401026
1,543,39,14,28,NaN,13-5-2021,27-5-2021,A-B,A,13-5-2021,...,110.0,9921.0,0.0,76.0,8078.260000,105104,111106,119185,82917,401026
2,543,39,14,28,NaN,13-5-2021,27-5-2021,A-B,A,13-5-2021,...,110.0,9600.0,0.0,80.0,4371.809999,122188,128190,132563,82917,401026
3,543,39,14,28,NaN,13-5-2021,27-5-2021,A-B,A,13-5-2021,...,110.0,6313.0,0.0,74.0,4598.900001,135564,141566,146166,82917,401026
4,543,39,14,28,NaN,13-5-2021,27-5-2021,A-B,A,13-5-2021,...,110.0,2165.0,100.0,38.0,12267.140000,149167,155169,167437,82917,401026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15813,644,34,14,23,1.0,13-5-2021,22-5-2021,B-A,B,13-5-2021,...,285.0,7481.0,92.0,50.0,3818.935000,598846,604852,608683,322528,668784
15814,644,34,14,23,1.0,13-5-2021,22-5-2021,B-A,B,13-5-2021,...,285.0,9911.0,8.0,86.0,7039.070000,611722,617728,624779,322528,668784
15815,644,34,14,23,1.0,13-5-2021,22-5-2021,B-A,B,13-5-2021,...,285.0,2070.0,87.0,15.0,5780.515000,627819,633828,639620,322528,668784
15816,644,34,14,23,1.0,13-5-2021,22-5-2021,B-A,B,13-5-2021,...,285.0,6312.0,0.0,99.0,5493.600000,642653,648661,654164,322528,668784


In [37]:
len(dataset)/44

359.5

In [38]:
os.chdir(os.path.join(data_dir,date_dir))
dataset.to_csv('allPMSdata.csv')

In [39]:
dataset.loc[:,'avg_PD_left'] = 0

In [40]:
dataset.ID == 192

0        False
1        False
2        False
3        False
4        False
         ...  
15813    False
15814    False
15815    False
15816    False
15817    False
Name: ID, Length: 15818, dtype: bool